In [ ]:
!pip install q scipy==1.1.0
import numpy as np
import torch 
import matplotlib.pyplot as plt
import torchvision
from time import time 
from torchvision import datasets, transforms
from torch import nn,optim 
import torch.nn.functional as F
from scipy.sparse.linalg import lsqr
from scipy.sparse import csr_matrix
from scipy.sparse import random
from scipy.sparse import rand
from scipy import stats
from torch.autograd import Variable

     |████████████████████████████████| 31.2MB 99kB/s 
ERROR: umap-learn 0.4.3 has requirement scipy>=1.3.1, but you'll have scipy 1.1.0 which is incompatible.
ERROR: tensorflow 2.2.0 has requirement scipy==1.4.1; python_version >= "3", but you'll have scipy 1.1.0 which is incompatible.
ERROR: plotnine 0.6.0 has requirement scipy>=1.2.0, but you'll have scipy 1.1.0 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: scipy 1.4.1
    Uninstalling scipy-1.4.1:
      Successfully uninstalled scipy-1.4.1


In [ ]:
# Normalize training set together with augmentation
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')



Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [ ]:
cuda = torch.device('cuda')     # Default CUDA device

In [ ]:
#same network as CNN but with bypass layers for each block 
#padding same 
#set random seed for reproducibility
torch.manual_seed(0)
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 5,padding = (2,2))
        self.conv2 = nn.Conv2d(32, 32, 5,padding = (2,2))
        self.conv3 = nn.Conv2d(32, 64, 5,padding = (2,2))
        self.conv4 = nn.Conv2d(64, 64, 5,padding = (2,2))
        self.conv5 = nn.Conv2d(64, 128, 5,padding = (2,2))
        self.conv6 = nn.Conv2d(128, 128, 5,padding = (2,2))
        self.sc1 = nn.Conv2d(3,32,kernel_size = (1,1),stride = (1,1), bias = False)
        self.sc2 = nn.Conv2d(32,64,kernel_size = (1,1),stride = (1,1), bias = False)
        self.sc3 = nn.Conv2d(64,128,kernel_size = (1,1),stride = (1,1), bias = False)

        self.pool = nn.MaxPool2d(2, 2)
        # self.fc1 = nn.Linear(int(32768/64), 10)
        self.fc1 = nn.Linear(128*4*4,10)
        
        self.drpout = nn.Dropout(0.2)
        # self.out = nn.Softmax(dim = 2) #hidhaval

    def forward(self, x):
        shortcut = x
        x = self.conv1(x)
        x = self.drpout(self.pool(F.relu(self.conv2(x)))) + self.pool(self.sc1(shortcut))     #(32,28,28)-> (32,12,12)
      
        shortcut2 = x
        x = (F.relu(self.conv3(x)))                           #(32,12,12)->(64,8,8)
        x = self.drpout(self.pool(F.relu(self.conv4(x)))) + self.pool(self.sc2(shortcut2))     #(64,8,8) ->(64,2,2)
              
        shortcut3 = x
        x = (F.relu(self.conv5(x)))
        x = self.drpout(self.pool(F.relu(self.conv6(x)))) + self.pool(self.sc3(shortcut3))
                
        x = x.view(-1,128*4*4)
        # # x = F.relu(self.fc1(x))
        x = self.fc1(x)
        # # x = self.out(x) #hidhaval
        return x

net = Net()
net = net.to(cuda)
torch.save(net.state_dict(),'weights_i.pth')

In [ ]:
#training
criterion = nn.CrossEntropyLoss()

optimizer = optim.Adam(net.parameters(), lr=0.0001, weight_decay=0.001)
for epoch in range(35):  # loop over the dataset multiple times
    correct = 0
    total = 0
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs = inputs.to(cuda)
        # labelsOH = one_hot_embedding(labels,10).long()
        # labelsOH = labelsOH.to(cuda)
        labels = labels.to(cuda)
        # zero the parameter gradients
        optimizer.zero_grad()

#         # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()        

        # print statistics
        running_loss += loss.item()
    print("Epoch Number: "+str(epoch+1)+'Training Accuracy: %d %%' % (
    100 * correct / total))

    # print("Epoch Number: "+str(epoch+1)+" Loss is: "+str(running_loss / 60000))
    # print('[%d] loss: %.3f' %(epoch + 1), (running_loss / 60000))

#save learned weights
torch.save(net.state_dict(),'weights_f.pth')


Epoch Number: 1Training Accuracy: 42 %
Epoch Number: 2Training Accuracy: 55 %
Epoch Number: 3Training Accuracy: 61 %
Epoch Number: 4Training Accuracy: 65 %
Epoch Number: 5Training Accuracy: 68 %
Epoch Number: 6Training Accuracy: 71 %
Epoch Number: 7Training Accuracy: 73 %
Epoch Number: 8Training Accuracy: 75 %
Epoch Number: 9Training Accuracy: 77 %
Epoch Number: 10Training Accuracy: 78 %
Epoch Number: 11Training Accuracy: 80 %
Epoch Number: 12Training Accuracy: 81 %
Epoch Number: 13Training Accuracy: 82 %
Epoch Number: 14Training Accuracy: 83 %
Epoch Number: 15Training Accuracy: 84 %
Epoch Number: 16Training Accuracy: 86 %
Epoch Number: 17Training Accuracy: 87 %
Epoch Number: 18Training Accuracy: 88 %
Epoch Number: 19Training Accuracy: 88 %
Epoch Number: 20Training Accuracy: 89 %
Epoch Number: 21Training Accuracy: 90 %
Epoch Number: 22Training Accuracy: 91 %
Epoch Number: 23Training Accuracy: 92 %
Epoch Number: 24Training Accuracy: 92 %
Epoch Number: 25Training Accuracy: 92 %
Epoch Num

In [ ]:
#testing #35 epochs 
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images = images.to(cuda)
        labels = labels.to(cuda)
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))
acc = (correct/total)
np.save('model_accuracy_1.npy',acc)

Accuracy of the network on the 10000 test images: 77 %


In [ ]:
#vectorize wi
wi =torch.load('weights_i.pth')

wi1 = np.asarray(wi['conv1.weight'].cpu().resize(1,32*3*5*5))
wi2 = np.asarray(wi['conv2.weight'].cpu().resize(1,32*32*5*5))
wi3 = np.asarray(wi['conv3.weight'].cpu().resize(1,64*32*5*5))
wi4 = np.asarray(wi['conv4.weight'].cpu().resize(1,64*64*5*5))
wi5 = np.asarray(wi['conv5.weight'].cpu().resize(1,128*64*5*5))
wi6 = np.asarray(wi['conv6.weight'].cpu().resize(1,128*128*5*5))
wi7 = np.asarray(wi['sc1.weight'].cpu().resize(1,3*32*1*1))
wi8 = np.asarray(wi['sc2.weight'].cpu().resize(1,32*64*1*1))
wi9 = np.asarray(wi['sc3.weight'].cpu().resize(1,64*128*1*1))
wi10 = np.asarray(wi['fc1.weight'].cpu().resize(1,10*2048))  #size of the resulting image after the given layers of convolutions 

# wi1 = np.asarray(wi['conv1.weight'].resize(1,32*3*5*5))
# wi2 = np.asarray(wi['conv2.weight'].resize(1,32*32*5*5))
# wi3 = np.asarray(wi['conv3.weight'].resize(1,64*32*5*5))
# wi4 = np.asarray(wi['conv4.weight'].resize(1,64*64*5*5))
# wi5 = np.asarray(wi['conv5.weight'].resize(1,128*64*5*5))
# wi6 = np.asarray(wi['conv6.weight'].resize(1,128*128*5*5))
# wi7 = np.asarray(wi['sc1.weight'].resize(1,3*32*1*1))
# wi8 = np.asarray(wi['sc2.weight'].resize(1,32*64*1*1))
# wi9 = np.asarray(wi['sc3.weight'].resize(1,64*128*1*1))
# wi10 = np.asarray(wi['fc1.weight'].resize(1,10*2048))  #size of the resulting image after the given layers of convolutions 


wi_vec = np.concatenate((wi1,wi2,wi3,wi4,wi5,wi6,wi7,wi8,wi9,wi10),axis =1)
wi_vec = wi_vec.T
dim_w = wi_vec.shape[0]
wi_vec.ravel().squeeze()

/usr/local/lib/python3.6/dist-packages/torch/tensor.py:365: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")


array([-0.00086451,  0.06194317, -0.09503707, ..., -0.01812525,
       -0.0001511 ,  0.01437058], dtype=float32)

In [ ]:
#vectorize wf
wf  =torch.load('weights_f.pth')

wf1 = np.asarray(wf['conv1.weight'].cpu().resize(1,32*3*5*5))
wf2 = np.asarray(wf['conv2.weight'].cpu().resize(1,32*32*5*5))
wf3 = np.asarray(wf['conv3.weight'].cpu().resize(1,64*32*5*5))
wf4 = np.asarray(wf['conv4.weight'].cpu().resize(1,64*64*5*5))
wf5 = np.asarray(wf['conv5.weight'].cpu().resize(1,128*64*5*5))
wf6 = np.asarray(wf['conv6.weight'].cpu().resize(1,128*128*5*5))
wf7 = np.asarray(wf['sc1.weight'].cpu().resize(1,3*32*1*1))
wf8 = np.asarray(wf['sc2.weight'].cpu().resize(1,32*64*1*1))
wf9 = np.asarray(wf['sc3.weight'].cpu().resize(1,64*128*1*1))
wf10 = np.asarray(wf['fc1.weight'].cpu().resize(1,10*2048))  #size of the resulting image after the given layers of convolutions 

# wf1 = np.asarray(wf['conv1.weight'].resize(1,32*3*5*5))
# wf2 = np.asarray(wf['conv2.weight'].resize(1,32*32*5*5))
# wf3 = np.asarray(wf['conv3.weight'].resize(1,64*32*5*5))
# wf4 = np.asarray(wf['conv4.weight'].resize(1,64*64*5*5))
# wf5 = np.asarray(wf['conv5.weight'].resize(1,128*64*5*5))
# wf6 = np.asarray(wf['conv6.weight'].resize(1,128*128*5*5))
# wf7 = np.asarray(wf['sc1.weight'].resize(1,3*32*1*1))
# wf8 = np.asarray(wf['sc2.weight'].resize(1,32*64*1*1))
# wf9 = np.asarray(wf['sc3.weight'].resize(1,64*128*1*1))
# wf10 = np.asarray(wf['fc1.weight'].resize(1,10*2048))  #size of the resulting image after the given layers of convolutions 

wf_vec = np.concatenate((wf1,wf2,wf3,wf4,wf5,wf6,wf7,wf8,wf9,wf10),axis =1)
wf_vec = wf_vec.T
wf_vec.ravel().squeeze()

/usr/local/lib/python3.6/dist-packages/torch/tensor.py:365: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")


array([ 0.02612493,  0.10060304, -0.0961016 , ..., -0.01282699,
        0.02053127,  0.04284701], dtype=float32)

In [ ]:
dim_w

826816

In [ ]:
#A run1 
from scipy import sparse
rvs = stats.norm().rvs
den = 10/dim_w
num_rows_arr  = np.asarray([2,4,8,16,32,64,128,256,512,1024,2048,4096,8192, 16384,32768,65536,131072,262144,524228])
n_runs = 2
acc  = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_i = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_f = np.zeros((n_runs,num_rows_arr.shape[0]))

for i_out in range(n_runs):
  print("outer iteration number:",(i_out+1))
  A_large = sparse.random(dim_w,dim_w, format='csr', density=den, data_rvs = rvs )

  for i_in in range(num_rows_arr.shape[0]):
      n_rows = num_rows_arr[i_in]
      #minimize Z such that AZ = A(Wf-w_init)
      A = A_large[0:n_rows,:]
      w = np.zeros(dim_w)


      w_diff = csr_matrix(wf_vec - wi_vec)
      b_eq = sparse.csr_matrix.dot(A,w_diff)

      bn = b_eq.toarray()
      bn.reshape((-1,))
      
      sol = lsqr(A,bn)
      z = sol[0]
      z = np.asmatrix(z).T
      w = z+ wi_vec

      #test performance of this reinitialization
      w1= w[0:32*3*5*5]
      w1 = torch.from_numpy(w1).reshape(32,3,5,5)
      w2 = w[32*3*5*5:32*3*5*5+32*32*5*5]
      w2 = torch.from_numpy(w2).reshape(32,32,5,5)
      w3 = w[32*3*5*5+32*32*5*5:32*3*5*5+32*32*5*5+ 64*32*5*5]
      w3 = torch.from_numpy(w3).reshape(64,32,5,5)
      w4 = w[32*3*5*5+32*32*5*5+ 64*32*5*5: 32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5]
      w4 = torch.from_numpy(w4).reshape(64,64,5,5)
      w5 = w[32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5: 32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5+ 128*64*5*5]
      w5 = torch.from_numpy(w5).reshape(128,64,5,5)
      w6 = w[32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5 + 128*64*5*5 :32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5 + 128*64*5*5 + 128*128*5*5]
      w6 = torch.from_numpy(w6).reshape(128,128,5,5)
      w7 = w[32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5 + 128*64*5*5 + 128*128*5*5 : 32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5 + 128*64*5*5 + 128*128*5*5 + 3*32*1*1]
      w7 = torch.from_numpy(w7).reshape(32,3,1,1)
      w8 = w[32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5 + 128*64*5*5 + 128*128*5*5 + 3*32 : 32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5 + 128*64*5*5 + 128*128*5*5 + 3*32 + 32*64]
      w8 = torch.from_numpy(w8).reshape(64,32,1,1)
      w9 = w[32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5 + 128*64*5*5 + 128*128*5*5 + 3*32 + 32*64 : 32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5 + 128*64*5*5 + 128*128*5*5 + 3*32 + 32*64 + 64*128]
      w9 = torch.from_numpy(w9).reshape(128,64,1,1)
      w10 = w[32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5 + 128*64*5*5 + 128*128*5*5 + 3*32 + 32*64 + 64*128 : 32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5 + 128*64*5*5 + 128*128*5*5 + 3*32 + 32*64 + 64*128 + 2048*10]
      w10 = torch.from_numpy(w10).reshape(10,2048)
      
      wf =torch.load('weights_f.pth')
      wf['conv1.weight'] = (w1)
      wf['conv2.weight'] = (w2)
      wf['conv3.weight'] = (w3)
      wf['conv4.weight'] = (w4)
      wf['conv5.weight'] = (w5)
      wf['conv6.weight'] = (w6)
      wf['sc1.weight'] = (w7)
      wf['sc2.weight'] = (w8)
      wf['sc3.weight'] = (w9)
      wf['fc1.weight'] = (w10)
      # wf.to(cuda)
      torch.save(wf,'w_affex.pth')


      net.load_state_dict(torch.load('w_affex.pth'))
      #testing
      correct = 0
      total = 0
      with torch.no_grad():
          for data in testloader:
              images, labels = data
              images = images.to(cuda)
              labels = labels.to(cuda)
              outputs = net(images)
              _, predicted = torch.max(outputs.data, 1)
              total += labels.size(0)
              correct += (predicted == labels).sum().item()

      # print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))

      acc[i_out,i_in] = correct/total
      dist_i[i_out,i_in] = np.linalg.norm(w-wi_vec)
      dist_f[i_out,i_in] = np.linalg.norm(w-wf_vec)
      print("\n n_rows = {0} , Accuracy ={1}".format(n_rows,acc[i_out,i_in]))
      print("\n Dist from Wi = {0} , Dist from Wf ={1}".format(dist_i[i_out,i_in],dist_f[i_out,i_in])) 

outer iteration number: 1

 n_rows = 2 , Accuracy =0.1057

 Dist from Wi = 0.022661503261714996 , Dist from Wf =17.892362847814034

 n_rows = 4 , Accuracy =0.1065

 Dist from Wi = 0.03803852422948443 , Dist from Wf =17.892336764445133

 n_rows = 8 , Accuracy =0.1074

 Dist from Wi = 0.062270846149384454 , Dist from Wf =17.892268837779216

 n_rows = 16 , Accuracy =0.1083

 Dist from Wi = 0.08030904386082702 , Dist from Wf =17.892196966254136

 n_rows = 32 , Accuracy =0.1083

 Dist from Wi = 0.10788494690302544 , Dist from Wf =17.892051941016987

 n_rows = 64 , Accuracy =0.1071

 Dist from Wi = 0.145888868261336 , Dist from Wf =17.891782422663944

 n_rows = 128 , Accuracy =0.1086

 Dist from Wi = 0.22159716461986018 , Dist from Wf =17.89100490521146

 n_rows = 256 , Accuracy =0.1058

 Dist from Wi = 0.2957852343383269 , Dist from Wf =17.889932166355305

 n_rows = 512 , Accuracy =0.1059

 Dist from Wi = 0.43982299634493505 , Dist from Wf =17.886970608579823

 n_rows = 1024 , Accuracy =0.1

In [ ]:
np.save("accuracy_A.npy", acc)
np.save("distance_i_A.npy", dist_i)
np.save("distance_f_A.npy", dist_f)

In [ ]:
#A + I  run1 
from scipy import sparse
I = sparse.identity(dim_w)
rvs = stats.norm().rvs
den = 10/dim_w
num_rows_arr  = np.asarray([2,4,8,16,32,64,128,256,512,1024,2048,4096,8192, 16384,32768,65536,131072,262144,524228])
n_runs = 2
acc  = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_i = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_f = np.zeros((n_runs,num_rows_arr.shape[0]))

for i_out in range(n_runs):
  print("outer iteration number:",(i_out+1))
  A_large = sparse.random(dim_w,dim_w, format='csr', density=den, data_rvs = rvs )
  A_large = A_large + I
  for i_in in range(num_rows_arr.shape[0]):
      n_rows = num_rows_arr[i_in]
      #minimize Z such that AZ = A(Wf-w_init)
      A = A_large[0:n_rows,:]
      w = np.zeros(dim_w)


      w_diff = csr_matrix(wf_vec - wi_vec)
      b_eq = sparse.csr_matrix.dot(A,w_diff)

      bn = b_eq.toarray()
      bn.reshape((-1,))
      
      sol = lsqr(A,bn)
      z = sol[0]
      z = np.asmatrix(z).T
      w = z+ wi_vec

      #test performance of this reinitialization
      w1= w[0:32*3*5*5]
      w1 = torch.from_numpy(w1).reshape(32,3,5,5)
      w2 = w[32*3*5*5:32*3*5*5+32*32*5*5]
      w2 = torch.from_numpy(w2).reshape(32,32,5,5)
      w3 = w[32*3*5*5+32*32*5*5:32*3*5*5+32*32*5*5+ 64*32*5*5]
      w3 = torch.from_numpy(w3).reshape(64,32,5,5)
      w4 = w[32*3*5*5+32*32*5*5+ 64*32*5*5: 32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5]
      w4 = torch.from_numpy(w4).reshape(64,64,5,5)
      w5 = w[32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5: 32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5+ 128*64*5*5]
      w5 = torch.from_numpy(w5).reshape(128,64,5,5)
      w6 = w[32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5 + 128*64*5*5 :32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5 + 128*64*5*5 + 128*128*5*5]
      w6 = torch.from_numpy(w6).reshape(128,128,5,5)
      w7 = w[32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5 + 128*64*5*5 + 128*128*5*5 : 32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5 + 128*64*5*5 + 128*128*5*5 + 3*32*1*1]
      w7 = torch.from_numpy(w7).reshape(32,3,1,1)
      w8 = w[32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5 + 128*64*5*5 + 128*128*5*5 + 3*32 : 32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5 + 128*64*5*5 + 128*128*5*5 + 3*32 + 32*64]
      w8 = torch.from_numpy(w8).reshape(64,32,1,1)
      w9 = w[32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5 + 128*64*5*5 + 128*128*5*5 + 3*32 + 32*64 : 32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5 + 128*64*5*5 + 128*128*5*5 + 3*32 + 32*64 + 64*128]
      w9 = torch.from_numpy(w9).reshape(128,64,1,1)
      w10 = w[32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5 + 128*64*5*5 + 128*128*5*5 + 3*32 + 32*64 + 64*128 : 32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5 + 128*64*5*5 + 128*128*5*5 + 3*32 + 32*64 + 64*128 + 2048*10]
      w10 = torch.from_numpy(w10).reshape(10,2048)
      
      wf =torch.load('weights_f.pth')
      wf['conv1.weight'] = (w1)
      wf['conv2.weight'] = (w2)
      wf['conv3.weight'] = (w3)
      wf['conv4.weight'] = (w4)
      wf['conv5.weight'] = (w5)
      wf['conv6.weight'] = (w6)
      wf['sc1.weight'] = (w7)
      wf['sc2.weight'] = (w8)
      wf['sc3.weight'] = (w9)
      wf['fc1.weight'] = (w10)
      # wf.to(cuda)
      torch.save(wf,'w_affex.pth')


      net.load_state_dict(torch.load('w_affex.pth'))
      #testing
      correct = 0
      total = 0
      with torch.no_grad():
          for data in testloader:
              images, labels = data
              images = images.to(cuda)
              labels = labels.to(cuda)
              outputs = net(images)
              _, predicted = torch.max(outputs.data, 1)
              total += labels.size(0)
              correct += (predicted == labels).sum().item()

      # print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))

      acc[i_out,i_in] = correct/total
      dist_i[i_out,i_in] = np.linalg.norm(w-wi_vec)
      dist_f[i_out,i_in] = np.linalg.norm(w-wf_vec)
      print("\n n_rows = {0} , Accuracy ={1}".format(n_rows,acc[i_out,i_in]))
      print("\n Dist from Wi = {0} , Dist from Wf ={1}".format(dist_i[i_out,i_in],dist_f[i_out,i_in])) 

outer iteration number: 1

 n_rows = 2 , Accuracy =0.1086

 Dist from Wi = 0.020030345899572553 , Dist from Wf =17.89236598683378

 n_rows = 4 , Accuracy =0.1098

 Dist from Wi = 0.04799165609262199 , Dist from Wf =17.89231283603272

 n_rows = 8 , Accuracy =0.1068

 Dist from Wi = 0.05499214021966516 , Dist from Wf =17.89229268948111

 n_rows = 16 , Accuracy =0.1061

 Dist from Wi = 0.0740585336620104 , Dist from Wf =17.892223929830045

 n_rows = 32 , Accuracy =0.109

 Dist from Wi = 0.1247941073731383 , Dist from Wf =17.89194199220448

 n_rows = 64 , Accuracy =0.107

 Dist from Wi = 0.16370089880876387 , Dist from Wf =17.891628317097588

 n_rows = 128 , Accuracy =0.1077

 Dist from Wi = 0.22953024195363214 , Dist from Wf =17.890904887391027

 n_rows = 256 , Accuracy =0.1071

 Dist from Wi = 0.3340511948704433 , Dist from Wf =17.889258554236854

 n_rows = 512 , Accuracy =0.1066

 Dist from Wi = 0.46863897210003297 , Dist from Wf =17.886238825814978

 n_rows = 1024 , Accuracy =0.1092

 

In [ ]:
np.save("accuracy_AI.npy", acc)
np.save("distance_i_AI.npy", dist_i)
np.save("distance_f_AI.npy", dist_f)

In [ ]:
wfcnnvec.shape

(1, 796000)

In [ ]:
# I  run1 
from scipy import sparse
den = 10/dim_w
n_runs = 1
num_rows_arr  = np.asarray([2,4,8,16,32,64,128,256,512,1024,2048,4096,8192, 16384,32768,65536,131072,262144,524228,32*3*25,32*3*25+32*32*25,32*3*25+32*32*25+64*32*25,32*3*25+32*32*25+64*32*25+64*64*25,32*3*25+32*32*25+64*32*25+64*64*25+128*64*25,32*3*25+32*32*25+64*32*25+64*64*25+128*64*25+128*128*25,32*3*25+32*32*25+64*32*25+64*64*25+128*64*25+128*128*25+20480])
num_rows_arr = np.sort(num_rows_arr)
acc  = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_i = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_f = np.zeros((n_runs,num_rows_arr.shape[0]))

wfcnnvec = np.concatenate((wf1,wf2,wf3,wf4,wf5,wf6),axis =1) 
wfcnnvec = wfcnnvec.T

wicnnvec = np.concatenate((wi1,wi2,wi3,wi4,wi5,wi6),axis =1) 
wicnnvec = wicnnvec.T
for i_out in range(n_runs):
  print("outer iteration number:",(i_out+1))
  for i_in in range(num_rows_arr.shape[0]):
      n_rows = num_rows_arr[i_in]
      #minimize Z such that AZ = A(Wf-w_init)
      
      w = np.zeros(wfcnnvec.shape)
      w[0:n_rows] = wfcnnvec[0:n_rows]
      w[n_rows:] = wicnnvec[n_rows:]
      
      w = np.concatenate((w.T,wf7,wf8,wf9,wf10),axis = 1).T

      #test performance of this reinitialization
      w1= w[0:32*3*5*5]
      w1 = torch.from_numpy(w1).reshape(32,3,5,5)
      w2 = w[32*3*5*5:32*3*5*5+32*32*5*5]
      w2 = torch.from_numpy(w2).reshape(32,32,5,5)
      w3 = w[32*3*5*5+32*32*5*5:32*3*5*5+32*32*5*5+ 64*32*5*5]
      w3 = torch.from_numpy(w3).reshape(64,32,5,5)
      w4 = w[32*3*5*5+32*32*5*5+ 64*32*5*5: 32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5]
      w4 = torch.from_numpy(w4).reshape(64,64,5,5)
      w5 = w[32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5: 32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5+ 128*64*5*5]
      w5 = torch.from_numpy(w5).reshape(128,64,5,5)
      w6 = w[32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5 + 128*64*5*5 :32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5 + 128*64*5*5 + 128*128*5*5]
      w6 = torch.from_numpy(w6).reshape(128,128,5,5)
      w7 = w[32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5 + 128*64*5*5 + 128*128*5*5 : 32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5 + 128*64*5*5 + 128*128*5*5 + 3*32*1*1]
      w7 = torch.from_numpy(w7).reshape(32,3,1,1)
      w8 = w[32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5 + 128*64*5*5 + 128*128*5*5 + 3*32 : 32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5 + 128*64*5*5 + 128*128*5*5 + 3*32 + 32*64]
      w8 = torch.from_numpy(w8).reshape(64,32,1,1)
      w9 = w[32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5 + 128*64*5*5 + 128*128*5*5 + 3*32 + 32*64 : 32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5 + 128*64*5*5 + 128*128*5*5 + 3*32 + 32*64 + 64*128]
      w9 = torch.from_numpy(w9).reshape(128,64,1,1)
      w10 = w[32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5 + 128*64*5*5 + 128*128*5*5 + 3*32 + 32*64 + 64*128 : 32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5 + 128*64*5*5 + 128*128*5*5 + 3*32 + 32*64 + 64*128 + 2048*10]
      w10 = torch.from_numpy(w10).reshape(10,2048)
      
      wf =torch.load('weights_f.pth')
      wf['conv1.weight'] = (w1)
      wf['conv2.weight'] = (w2)
      wf['conv3.weight'] = (w3)
      wf['conv4.weight'] = (w4)
      wf['conv5.weight'] = (w5)
      wf['conv6.weight'] = (w6)
      wf['sc1.weight'] = (w7)
      wf['sc2.weight'] = (w8)
      wf['sc3.weight'] = (w9)
      wf['fc1.weight'] = (w10)
      # wf.to(cuda)
      torch.save(wf,'w_affex.pth')


      net.load_state_dict(torch.load('w_affex.pth'))
      #testing
      correct = 0
      total = 0
      with torch.no_grad():
          for data in testloader:
              images, labels = data
              images = images.to(cuda)
              labels = labels.to(cuda)
              outputs = net(images)
              _, predicted = torch.max(outputs.data, 1)
              total += labels.size(0)
              correct += (predicted == labels).sum().item()

      # print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))

      acc[i_out,i_in] = correct/total
      dist_i[i_out,i_in] = np.linalg.norm(w-wi_vec)
      dist_f[i_out,i_in] = np.linalg.norm(w-wf_vec)
      print("\n n_rows = {0} , Accuracy ={1}".format(n_rows,acc[i_out,i_in]))
      print("\n Dist from Wi = {0} , Dist from Wf ={1}".format(dist_i[i_out,i_in],dist_f[i_out,i_in])) 

outer iteration number: 1

 n_rows = 2 , Accuracy =0.2178

 Dist from Wi = 5.404142060941787 , Dist from Wf =17.05674090811971

 n_rows = 4 , Accuracy =0.2167

 Dist from Wi = 5.404144723804251 , Dist from Wf =17.05674006443621

 n_rows = 8 , Accuracy =0.2202

 Dist from Wi = 5.40438624574483 , Dist from Wf =17.056663540340143

 n_rows = 16 , Accuracy =0.2177

 Dist from Wi = 5.4047212576829144 , Dist from Wf =17.056557388532863

 n_rows = 32 , Accuracy =0.2226

 Dist from Wi = 5.404971574583156 , Dist from Wf =17.05647806844991

 n_rows = 64 , Accuracy =0.2222

 Dist from Wi = 5.405652067023658 , Dist from Wf =17.05626241448716

 n_rows = 128 , Accuracy =0.2194

 Dist from Wi = 5.417587351442686 , Dist from Wf =17.052475160839638

 n_rows = 256 , Accuracy =0.2193

 Dist from Wi = 5.426221062229268 , Dist from Wf =17.04972981619897

 n_rows = 512 , Accuracy =0.2175

 Dist from Wi = 5.4490272111803595 , Dist from Wf =17.042454760784334

 n_rows = 1024 , Accuracy =0.2173

 Dist from Wi =

In [ ]:
np.save("accuracy_I.npy", acc)
np.save("distance_i_I.npy", dist_i)
np.save("distance_f_I.npy", dist_f)

In [ ]:
# Irev  run1 
from scipy import sparse
den = 10/dim_w
n_runs = 1
num_rows_arr  = np.asarray([2,4,8,16,32,64,128,256,512,1024,2048,4096,8192, 16384,32768,65536,131072,262144,524228,32*3*25,32*3*25+32*32*25,32*3*25+32*32*25+64*32*25,32*3*25+32*32*25+64*32*25+64*64*25,32*3*25+32*32*25+64*32*25+64*64*25+128*64*25,32*3*25+32*32*25+64*32*25+64*64*25+128*64*25+128*128*25,32*3*25+32*32*25+64*32*25+64*64*25+128*64*25+128*128*25+20480])
num_rows_arr = np.sort(num_rows_arr)
acc  = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_i = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_f = np.zeros((n_runs,num_rows_arr.shape[0]))

wfcnnvec = np.concatenate((wf1,wf2,wf3,wf4,wf5,wf6),axis =1) 
wfcnnvec = wfcnnvec.T

wicnnvec = np.concatenate((wi1,wi2,wi3,wi4,wi5,wi6),axis =1) 
wicnnvec = wicnnvec.T
for i_out in range(n_runs):
  print("outer iteration number:",(i_out+1))
  for i_in in range(num_rows_arr.shape[0]):
      n_rows = num_rows_arr[i_in]
      #minimize Z such that AZ = A(Wf-w_init)
      
      w = np.zeros(wfcnnvec.shape)
      w[0:-n_rows] = wicnnvec[0:-n_rows]
      w[-n_rows:] = wfcnnvec[-n_rows:]
      
      w = np.concatenate((w.T,wf7,wf8,wf9,wf10),axis = 1).T

      #test performance of this reinitialization
      w1= w[0:32*3*5*5]
      w1 = torch.from_numpy(w1).reshape(32,3,5,5)
      w2 = w[32*3*5*5:32*3*5*5+32*32*5*5]
      w2 = torch.from_numpy(w2).reshape(32,32,5,5)
      w3 = w[32*3*5*5+32*32*5*5:32*3*5*5+32*32*5*5+ 64*32*5*5]
      w3 = torch.from_numpy(w3).reshape(64,32,5,5)
      w4 = w[32*3*5*5+32*32*5*5+ 64*32*5*5: 32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5]
      w4 = torch.from_numpy(w4).reshape(64,64,5,5)
      w5 = w[32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5: 32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5+ 128*64*5*5]
      w5 = torch.from_numpy(w5).reshape(128,64,5,5)
      w6 = w[32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5 + 128*64*5*5 :32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5 + 128*64*5*5 + 128*128*5*5]
      w6 = torch.from_numpy(w6).reshape(128,128,5,5)
      w7 = w[32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5 + 128*64*5*5 + 128*128*5*5 : 32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5 + 128*64*5*5 + 128*128*5*5 + 3*32*1*1]
      w7 = torch.from_numpy(w7).reshape(32,3,1,1)
      w8 = w[32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5 + 128*64*5*5 + 128*128*5*5 + 3*32 : 32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5 + 128*64*5*5 + 128*128*5*5 + 3*32 + 32*64]
      w8 = torch.from_numpy(w8).reshape(64,32,1,1)
      w9 = w[32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5 + 128*64*5*5 + 128*128*5*5 + 3*32 + 32*64 : 32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5 + 128*64*5*5 + 128*128*5*5 + 3*32 + 32*64 + 64*128]
      w9 = torch.from_numpy(w9).reshape(128,64,1,1)
      w10 = w[32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5 + 128*64*5*5 + 128*128*5*5 + 3*32 + 32*64 + 64*128 : 32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5 + 128*64*5*5 + 128*128*5*5 + 3*32 + 32*64 + 64*128 + 2048*10]
      w10 = torch.from_numpy(w10).reshape(10,2048)
      
      wf =torch.load('weights_f.pth')
      wf['conv1.weight'] = (w1)
      wf['conv2.weight'] = (w2)
      wf['conv3.weight'] = (w3)
      wf['conv4.weight'] = (w4)
      wf['conv5.weight'] = (w5)
      wf['conv6.weight'] = (w6)
      wf['sc1.weight'] = (w7)
      wf['sc2.weight'] = (w8)
      wf['sc3.weight'] = (w9)
      wf['fc1.weight'] = (w10)
      # wf.to(cuda)
      torch.save(wf,'w_affex.pth')


      net.load_state_dict(torch.load('w_affex.pth'))
      #testing
      correct = 0
      total = 0
      with torch.no_grad():
          for data in testloader:
              images, labels = data
              images = images.to(cuda)
              labels = labels.to(cuda)
              outputs = net(images)
              _, predicted = torch.max(outputs.data, 1)
              total += labels.size(0)
              correct += (predicted == labels).sum().item()

      # print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))

      acc[i_out,i_in] = correct/total
      dist_i[i_out,i_in] = np.linalg.norm(w-wi_vec)
      dist_f[i_out,i_in] = np.linalg.norm(w-wf_vec)
      print("\n n_rows = {0} , Accuracy ={1}".format(n_rows,acc[i_out,i_in]))
      print("\n Dist from Wi = {0} , Dist from Wf ={1}".format(dist_i[i_out,i_in],dist_f[i_out,i_in])) 

outer iteration number: 1

 n_rows = 2 , Accuracy =0.2155

 Dist from Wi = 5.403985805599832 , Dist from Wf =17.056790414214515

 n_rows = 4 , Accuracy =0.2181

 Dist from Wi = 5.403995190663127 , Dist from Wf =17.056787440806502

 n_rows = 8 , Accuracy =0.2153

 Dist from Wi = 5.404057409099043 , Dist from Wf =17.056767728404065

 n_rows = 16 , Accuracy =0.2184

 Dist from Wi = 5.40419677616071 , Dist from Wf =17.056723572424417

 n_rows = 32 , Accuracy =0.2169

 Dist from Wi = 5.404697909150777 , Dist from Wf =17.05656478697823

 n_rows = 64 , Accuracy =0.2179

 Dist from Wi = 5.405372242461321 , Dist from Wf =17.05635109693727

 n_rows = 128 , Accuracy =0.2202

 Dist from Wi = 5.407518967325149 , Dist from Wf =17.055670624152643

 n_rows = 256 , Accuracy =0.2205

 Dist from Wi = 5.41041871762248 , Dist from Wf =17.054750983862675

 n_rows = 512 , Accuracy =0.2216

 Dist from Wi = 5.417155962777223 , Dist from Wf =17.052612207415958

 n_rows = 1024 , Accuracy =0.2154

 Dist from Wi =

In [ ]:
np.save("accuracy_UI.npy", acc)
np.save("distance_i_UI.npy", dist_i)
np.save("distance_f_UI.npy", dist_f)

In [ ]:
from google.colab import files
files.download('accuracy_UI.npy') 
files.download('distance_i_UI.npy')
files.download('distance_f_UI.npy')

files.download('accuracy_I.npy') 
files.download('distance_i_I.npy')
files.download('distance_f_I.npy')

files.download('accuracy_AI.npy') 
files.download('distance_i_AI.npy')
files.download('distance_f_AI.npy')

files.download('accuracy_A.npy') 
files.download('distance_i_A.npy')
files.download('distance_f_A.npy')

In [ ]:
# I  run1 
from scipy import sparse
I = sparse.identity(dim_w)
rvs = stats.norm().rvs
den = 10/dim_w
num_rows_arr  = np.asarray([2,4,8,16,32,64,128,256,512,1024,2048,4096,8192, 16384,32768,65536,131072,262144,524228])
n_runs = 1
acc  = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_i = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_f = np.zeros((n_runs,num_rows_arr.shape[0]))

for i_out in range(n_runs):
  print("outer iteration number:",(i_out+1))
  A_large = I
  for i_in in range(num_rows_arr.shape[0]):
      n_rows = num_rows_arr[i_in]
      #minimize Z such that AZ = A(Wf-w_init)
      A = A_large.tocsr()[0:n_rows,:]
      w = np.zeros(dim_w)


      w_diff = csr_matrix(wf_vec - wi_vec)
      b_eq = sparse.csr_matrix.dot(A,w_diff)

      bn = b_eq.toarray()
      bn.reshape((-1,))
      
      sol = lsqr(A,bn)
      z = sol[0]
      z = np.asmatrix(z).T
      w = z+ wi_vec

      #test performance of this reinitialization
      w1= w[0:32*3*5*5]
      w1 = torch.from_numpy(w1).reshape(32,3,5,5)
      w2 = w[32*3*5*5:32*3*5*5+32*32*5*5]
      w2 = torch.from_numpy(w2).reshape(32,32,5,5)
      w3 = w[32*3*5*5+32*32*5*5:32*3*5*5+32*32*5*5+ 64*32*5*5]
      w3 = torch.from_numpy(w3).reshape(64,32,5,5)
      w4 = w[32*3*5*5+32*32*5*5+ 64*32*5*5: 32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5]
      w4 = torch.from_numpy(w4).reshape(64,64,5,5)
      w5 = w[32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5: 32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5+ 128*64*5*5]
      w5 = torch.from_numpy(w5).reshape(128,64,5,5)
      w6 = w[32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5 + 128*64*5*5 :32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5 + 128*64*5*5 + 128*128*5*5]
      w6 = torch.from_numpy(w6).reshape(128,128,5,5)
      w7 = w[32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5 + 128*64*5*5 + 128*128*5*5 : 32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5 + 128*64*5*5 + 128*128*5*5 + 3*32*1*1]
      w7 = torch.from_numpy(w7).reshape(32,3,1,1)
      w8 = w[32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5 + 128*64*5*5 + 128*128*5*5 + 3*32 : 32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5 + 128*64*5*5 + 128*128*5*5 + 3*32 + 32*64]
      w8 = torch.from_numpy(w8).reshape(64,32,1,1)
      w9 = w[32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5 + 128*64*5*5 + 128*128*5*5 + 3*32 + 32*64 : 32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5 + 128*64*5*5 + 128*128*5*5 + 3*32 + 32*64 + 64*128]
      w9 = torch.from_numpy(w9).reshape(128,64,1,1)
      w10 = w[32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5 + 128*64*5*5 + 128*128*5*5 + 3*32 + 32*64 + 64*128 : 32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5 + 128*64*5*5 + 128*128*5*5 + 3*32 + 32*64 + 64*128 + 2048*10]
      w10 = torch.from_numpy(w10).reshape(10,2048)
      
      wf =torch.load('weights_f.pth')
      wf['conv1.weight'] = (w1)
      wf['conv2.weight'] = (w2)
      wf['conv3.weight'] = (w3)
      wf['conv4.weight'] = (w4)
      wf['conv5.weight'] = (w5)
      wf['conv6.weight'] = (w6)
      wf['sc1.weight'] = (w7)
      wf['sc2.weight'] = (w8)
      wf['sc3.weight'] = (w9)
      wf['fc1.weight'] = (w10)
      # wf.to(cuda)
      torch.save(wf,'w_affex.pth')


      net.load_state_dict(torch.load('w_affex.pth'))
      #testing
      correct = 0
      total = 0
      with torch.no_grad():
          for data in testloader:
              images, labels = data
              images = images.to(cuda)
              labels = labels.to(cuda)
              outputs = net(images)
              _, predicted = torch.max(outputs.data, 1)
              total += labels.size(0)
              correct += (predicted == labels).sum().item()

      # print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))

      acc[i_out,i_in] = correct/total
      dist_i[i_out,i_in] = np.linalg.norm(w-wi_vec)
      dist_f[i_out,i_in] = np.linalg.norm(w-wf_vec)
      print("\n n_rows = {0} , Accuracy ={1}".format(n_rows,acc[i_out,i_in]))
      print("\n Dist from Wi = {0} , Dist from Wf ={1}".format(dist_i[i_out,i_in],dist_f[i_out,i_in])) 

In [ ]:
np.save("accuracy_I1.npy", acc)
np.save("distance_i_I1.npy", dist_i)
np.save("distance_f_I1.npy", dist_f)

In [ ]:
#I UI experiment

In [ ]:
!pip install q scipy==1.1.0
import numpy as np
import torch 
import matplotlib.pyplot as plt
import torchvision
from time import time 
from torchvision import datasets, transforms
from torch import nn,optim 
import torch.nn.functional as F
from scipy.sparse.linalg import lsqr
from scipy.sparse import csr_matrix
from scipy.sparse import random
from scipy.sparse import rand
from scipy import stats
from torch.autograd import Variable

     |████████████████████████████████| 31.2MB 103kB/s 
ERROR: umap-learn 0.4.3 has requirement scipy>=1.3.1, but you'll have scipy 1.1.0 which is incompatible.
ERROR: tensorflow 2.2.0 has requirement scipy==1.4.1; python_version >= "3", but you'll have scipy 1.1.0 which is incompatible.
ERROR: plotnine 0.6.0 has requirement scipy>=1.2.0, but you'll have scipy 1.1.0 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: scipy 1.4.1
    Uninstalling scipy-1.4.1:
      Successfully uninstalled scipy-1.4.1


In [ ]:
# Normalize training set together with augmentation
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')



Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [ ]:
#define CNN source = https://zhenye-na.github.io/2018/09/28/pytorch-cnn-cifar10.html
#padding same 
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 5,padding = (2,2))
        self.conv2 = nn.Conv2d(32, 32, 5,padding = (2,2))
        self.conv3 = nn.Conv2d(32, 64, 5,padding = (2,2))
        self.conv4 = nn.Conv2d(64, 64, 5,padding = (2,2))
        self.conv5 = nn.Conv2d(64, 128, 5,padding = (2,2))
        self.conv6 = nn.Conv2d(128, 128, 5,padding = (2,2))
        self.sc1 = nn.Conv2d(3,32,kernel_size = (1,1),stride = (1,1), bias = False)
        self.sc2 = nn.Conv2d(32,64,kernel_size = (1,1),stride = (1,1), bias = False)
        self.sc3 = nn.Conv2d(64,128,kernel_size = (1,1),stride = (1,1), bias = False)

        self.pool = nn.MaxPool2d(2, 2)
        # self.fc1 = nn.Linear(int(32768/64), 10)
        self.fc1 = nn.Linear(128*4*4,10)
        
        self.drpout = nn.Dropout(0.2)
        # self.out = nn.Softmax(dim = 2) #hidhaval

    def forward(self, x):
        shortcut = x
        x = self.conv1(x)
        x = self.drpout(self.pool(F.relu(self.conv2(x)))) + self.pool(self.sc1(shortcut))     #(32,28,28)-> (32,12,12)
      
        shortcut2 = x
        x = (F.relu(self.conv3(x)))                           #(32,12,12)->(64,8,8)
        x = self.drpout(self.pool(F.relu(self.conv4(x)))) + self.pool(self.sc2(shortcut2))     #(64,8,8) ->(64,2,2)
              
        shortcut3 = x
        x = (F.relu(self.conv5(x)))
        x = self.drpout(self.pool(F.relu(self.conv6(x)))) + self.pool(self.sc3(shortcut3))
                
        x = x.view(-1,128*4*4)
        # # x = F.relu(self.fc1(x))
        x = self.fc1(x)
        # # x = self.out(x) #hidhaval
        return x

net = Net()
# net = net.to(cuda)
torch.save(net.state_dict(),'weights_i.pth')

In [ ]:
criterion = nn.CrossEntropyLoss()

optimizer = optim.Adam(net.parameters(), lr=0.0001, weight_decay=0.001)
for epoch in range(35):  # loop over the dataset multiple times
    correct = 0
    total = 0
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        # inputs = inputs.to(cuda)
        # labelsOH = one_hot_embedding(labels,10).long()
        # labelsOH = labelsOH.to(cuda)
        # labels = labels.to(cuda)
        # zero the parameter gradients
        optimizer.zero_grad()

#         # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()        

        # print statistics
        running_loss += loss.item()
    print("Epoch Number: "+str(epoch+1)+'Training Accuracy: %d %%' % (
    100 * correct / total))

    # print("Epoch Number: "+str(epoch+1)+" Loss is: "+str(running_loss / 60000))
    # print('[%d] loss: %.3f' %(epoch + 1), (running_loss / 60000))

#save learned weights
torch.save(net.state_dict(),'weights_f.pth')


KeyboardInterrupt: ignored

In [ ]:
#testing #35 epochs 
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        # images = images.to(cuda)
        # labels = labels.to(cuda)
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))
acc = (correct/total)
np.save('model_accuracy_1.npy',acc)

In [ ]:
#vectorize wi
wi =torch.load('weights_i.pth')

wi1 = np.asarray(wi['conv1.weight'].resize(1,32*3*5*5))
wi2 = np.asarray(wi['conv2.weight'].resize(1,32*32*5*5))
wi3 = np.asarray(wi['conv3.weight'].resize(1,64*32*5*5))
wi4 = np.asarray(wi['conv4.weight'].resize(1,64*64*5*5))
wi5 = np.asarray(wi['conv5.weight'].resize(1,128*64*5*5))
wi6 = np.asarray(wi['conv6.weight'].resize(1,128*128*5*5))
wi7 = np.asarray(wi['sc1.weight'].resize(1,3*32*1*1))
wi8 = np.asarray(wi['sc2.weight'].resize(1,32*64*1*1))
wi9 = np.asarray(wi['sc3.weight'].resize(1,64*128*1*1))
wi10 = np.asarray(wi['fc1.weight'].resize(1,10*2048))  #size of the resulting image after the given layers of convolutions 

wicnnvec = np.concatenate((wi1,wi2,wi3,wi4,wi5,wi6),axis =1) 
wi_vec = np.concatenate((wi1,wi2,wi3,wi4,wi5,wi6,wi7,wi8,wi9,wi10),axis =1)
wi_vec = wi_vec.T
dim_w = wi_vec.shape[0]
wi_vec.ravel().squeeze()

In [ ]:
#vectorize wf
wf  =torch.load('weights_f.pth')

wf1 = np.asarray(wf['conv1.weight'].resize(1,32*3*5*5))
wf2 = np.asarray(wf['conv2.weight'].resize(1,32*32*5*5))
wf3 = np.asarray(wf['conv3.weight'].resize(1,64*32*5*5))
wf4 = np.asarray(wf['conv4.weight'].resize(1,64*64*5*5))
wf5 = np.asarray(wf['conv5.weight'].resize(1,128*64*5*5))
wf6 = np.asarray(wf['conv6.weight'].resize(1,128*128*5*5))
wf7 = np.asarray(wf['sc1.weight'].resize(1,3*32*1*1))
wf8 = np.asarray(wf['sc2.weight'].resize(1,32*64*1*1))
wf9 = np.asarray(wf['sc3.weight'].resize(1,64*128*1*1))
wf10 = np.asarray(wf['fc1.weight'].resize(1,10*2048))  #size of the resulting image after the given layers of convolutions 

wfcnnvec = np.concatenate((wf1,wf2,wf3,wf4,wf5,wf6),axis =1) 
wf_vec = np.concatenate((wf1,wf2,wf3,wf4,wf5,wf6,wf7,wf8,wf9,wf10),axis =1)
wf_vec = wf_vec.T
wf_vec.ravel().squeeze()

In [ ]:
# I  run1 
from scipy import sparse
num_rows_arr  = np.asarray([2,4,8,16,32,64,128,256,512,1024,2048,4096,8192, 16384,32768,65536,131072,262144,524228,32*3*25,32*3*25+32*32*25,32*3*25+32*32*25+64*32*25,32*3*25+32*32*25+64*32*25+64*64*25,32*3*25+32*32*25+64*32*25+64*64*25+128*64*25,32*3*25+32*32*25+64*32*25+64*64*25+128*64*25+128*128*25,32*3*25+32*32*25+64*32*25+64*64*25+128*64*25+128*128*25+20480])
num_rows_arr = np.sort(num_rows_arr)
acc  = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_i = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_f = np.zeros((n_runs,num_rows_arr.shape[0]))

for i_out in range(n_runs):
  print("outer iteration number:",(i_out+1))
  for i_in in range(num_rows_arr.shape[0]):
      n_rows = num_rows_arr[i_in]
      #minimize Z such that AZ = A(Wf-w_init)
      
      w = np.zeros(wfcnnvec.shape)
      w[0:n_rows] = wfcnn[0:n_rows]
      w[n_rows:] = wicnn[n_rows:]
      
      w = np.concatenate((w,w7,w8,w9,w10),axis = 1)
      #test performance of this reinitialization
      w1= w[0:32*3*5*5]
      w1 = torch.from_numpy(w1).reshape(32,3,5,5)
      w2 = w[32*3*5*5:32*3*5*5+32*32*5*5]
      w2 = torch.from_numpy(w2).reshape(32,32,5,5)
      w3 = w[32*3*5*5+32*32*5*5:32*3*5*5+32*32*5*5+ 64*32*5*5]
      w3 = torch.from_numpy(w3).reshape(64,32,5,5)
      w4 = w[32*3*5*5+32*32*5*5+ 64*32*5*5: 32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5]
      w4 = torch.from_numpy(w4).reshape(64,64,5,5)
      w5 = w[32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5: 32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5+ 128*64*5*5]
      w5 = torch.from_numpy(w5).reshape(128,64,5,5)
      w6 = w[32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5 + 128*64*5*5 :32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5 + 128*64*5*5 + 128*128*5*5]
      w6 = torch.from_numpy(w6).reshape(128,128,5,5)
      w7 = w[32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5 + 128*64*5*5 + 128*128*5*5 : 32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5 + 128*64*5*5 + 128*128*5*5 + 3*32*1*1]
      w7 = torch.from_numpy(w7).reshape(32,3,1,1)
      w8 = w[32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5 + 128*64*5*5 + 128*128*5*5 + 3*32 : 32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5 + 128*64*5*5 + 128*128*5*5 + 3*32 + 32*64]
      w8 = torch.from_numpy(w8).reshape(64,32,1,1)
      w9 = w[32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5 + 128*64*5*5 + 128*128*5*5 + 3*32 + 32*64 : 32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5 + 128*64*5*5 + 128*128*5*5 + 3*32 + 32*64 + 64*128]
      w9 = torch.from_numpy(w9).reshape(128,64,1,1)
      w10 = w[32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5 + 128*64*5*5 + 128*128*5*5 + 3*32 + 32*64 + 64*128 : 32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5 + 128*64*5*5 + 128*128*5*5 + 3*32 + 32*64 + 64*128 + 2048*10]
      w10 = torch.from_numpy(w10).reshape(10,2048)
      
      wf =torch.load('weights_f.pth')
      wf['conv1.weight'] = (w1)
      wf['conv2.weight'] = (w2)
      wf['conv3.weight'] = (w3)
      wf['conv4.weight'] = (w4)
      wf['conv5.weight'] = (w5)
      wf['conv6.weight'] = (w6)
      wf['sc1.weight'] = (w7)
      wf['sc2.weight'] = (w8)
      wf['sc3.weight'] = (w9)
      wf['fc1.weight'] = (w10)
      # wf.to(cuda)
      torch.save(wf,'w_affex.pth')


      net.load_state_dict(torch.load('w_affex.pth'))
      #testing
      correct = 0
      total = 0
      with torch.no_grad():
          for data in testloader:
              images, labels = data
              images = images.to(cuda)
              labels = labels.to(cuda)
              outputs = net(images)
              _, predicted = torch.max(outputs.data, 1)
              total += labels.size(0)
              correct += (predicted == labels).sum().item()

      # print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))

      acc[i_out,i_in] = correct/total
      dist_i[i_out,i_in] = np.linalg.norm(w-wi_vec)
      dist_f[i_out,i_in] = np.linalg.norm(w-wf_vec)
      print("\n n_rows = {0} , Accuracy ={1}".format(n_rows,acc[i_out,i_in]))
      print("\n Dist from Wi = {0} , Dist from Wf ={1}".format(dist_i[i_out,i_in],dist_f[i_out,i_in])) 

In [ ]:
826816